# Species-level relative abundance in response to regenerating harvests 

```yaml
---
title: "Species-level Relative Abundance in Response to Regenerating Harvests"
description: >
  This notebook implements negative binomial models to analyze species-level 
  relative abundance in response to regenerating forest harvests. It covers 
  data loading and preparation, model fitting using Stan via CmdStanPy, 
  and model comparison using WAIC and LOO. The analysis focuses on bird 
  species abundance in relation to various environmental covariates such as 
  vegetation type, patch characteristics, and forest age.
author: "Isabelle Lebeuf-Taylor"
date: "2024-09-25"
tags:
  - ecological modeling
  - bird abundance
  - Stan
  - negative binomial
---
```

## A. Load data and packages

In [1]:
import os
import pandas as pd
from cmdstanpy import CmdStanModel
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from arviz.labels import BaseLabeller

c:\Users\ilebe\anaconda3\envs\stan\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### A. Load data

In [11]:
truncated_150m_counts = pd.read_csv("Data/Abundance within 150m.csv")
truncated_250m_counts = pd.read_csv("Data/Abundance within 250m.csv")
untruncated_counts = pd.read_csv("Data/Abundance untruncated distance.csv")
covariates = pd.read_csv("Data/covariates.csv")

# B. Prepare data

### B.1 Load data

In [12]:
df_sites_to_use = pd.read_csv(r"C:\Users\ilebe\Documents\!Masters!\Chapter 1 - Single species and retention\Bird-abundance-limited-perceptibility\Data\sites_to_use.csv")
list_sites_to_use = df_sites_to_use.sites.to_list()
covariates = covariates.loc[covariates['location'].isin(list_sites_to_use)]
truncated_150m_counts = truncated_150m_counts.loc[truncated_150m_counts['location'].isin(list_sites_to_use)]
truncated_250m_counts = truncated_250m_counts.loc[truncated_250m_counts['location'].isin(list_sites_to_use)]
untruncated_counts = untruncated_counts.loc[untruncated_counts['location'].isin(list_sites_to_use)]

In [14]:
covariates.to_csv(r"C:\Users\ilebe\Documents\!Masters!\Chapter 1 - Single species and retention\Bird-abundance-limited-perceptibility\Data\Covariates.csv")
truncated_150m_counts.to_csv(r"C:\Users\ilebe\Documents\!Masters!\Chapter 1 - Single species and retention\Bird-abundance-limited-perceptibility\Data\Abundance within 150m.csv")
truncated_250m_counts.to_csv(r"C:\Users\ilebe\Documents\!Masters!\Chapter 1 - Single species and retention\Bird-abundance-limited-perceptibility\Data\Abundance within 250m.csv")
untruncated_counts.to_csv(r"C:\Users\ilebe\Documents\!Masters!\Chapter 1 - Single species and retention\Bird-abundance-limited-perceptibility\Data\Abundance untruncated distance.csv")

### B.2 Prepare data, per response dataset

In [ ]:
# 1. Choose the response dataset
# obs_count_10 = truncated_150m_counts
# obs_count_10 = truncated_250m_counts
obs_count_10 = untruncated_counts

# 2. Randomly select 10 transcribed recordings per site
final_obs_count = obs_count_10.groupby('location').apply(lambda x: x.sample(n=10, replace=False, random_state = 123)).reset_index(drop=True)
final_obs_count['visit_number'] = final_obs_count.groupby('location').cumcount() + 1

# 3. Assigned integers to veg categories
covariates['Tree_group'] =  covariates['Veg_cat'].map({"Pine": 1, "Deciduous": 2, "Mixedwood": 3, "Spruce": 4})

# 4. Make a  patch presence column
covariates['patch'] = covariates.apply(lambda row: 0 if row['RETN_m2'] == 0 else 1, axis=1)

# 5. Calculate the area to edge ratio
covariates['edge'] = covariates['RETN_m2'] / covariates['RETN_perimeter_m']
covariates['edge'] = covariates['edge'].fillna(0)

### B.3 Prepare data list for Stan models, per bird

In [ ]:
# 1. Choose your bird
bird = 'WTSP'
# All species  = ['WTSP', 'YRWA', 'RCKI', 'TEWA', 'REVI']

# 2. Prepare lists
N_matrix_visit_num = final_obs_count.pivot(index='location', columns='visit_number', values=f"{bird}").fillna(0).astype(int).values.tolist()
veg_categories_ints = covariates['Tree_group'].to_list()
dist_forest = covariates['Dist_near_forest'].to_list()
size = covariates['RETN_m2'].tolist()
patch = covariates['patch'].to_list()
edge = covariates['edge'].to_list()
age = covariates['Year_since_logging'].tolist()
age2 = [i**2 for i in age]

# 3. Calculate the maximum count encountered at each site
max_counts = [max(site_counts) for site_counts in N_matrix_visit_num]

# 4. Calculate the maximum count total
max_total = max(max_counts)

### B.4 Compile data list into a dictionary for the CmdStanPy interfce of Stan

In [ ]:
obs_and_covs_dict_max_count = {
    "I": len(obs_count_10['location'].unique()),
    "J": 10,
    "M": max_counts,
    "tree_groups": veg_categories_ints,
    "dist_forest" : dist_forest,
    "size": size,
    "patch" : patch,
    "edge" : edge,
    "age": age,
    "age2" :age2,
}

# Scale covariates between 0 and 1

MinMax_obs_and_covs_dict = {}
for key, value in obs_and_covs_dict_max_count.items():
    if isinstance(value, (list, np.ndarray)) and key not in ['M', 'patch', "tree_groups"]:
        scaled = MinMaxScaler().fit_transform(np.array(value).reshape(-1, 1))
        MinMax_obs_and_covs_dict[key] = scaled.ravel().tolist()

    else:
        MinMax_obs_and_covs_dict[key] = value


### B.5 Check dispersion of the observations

In [ ]:
observed_data = MinMax_obs_and_covs_dict.observed_data['M'].values

# Calculate mean and variance
mean_observed = np.mean(observed_data)
variance_observed = np.var(observed_data)

# C. Run models

### C.1 Load stan files

> Adjust covariates in the stan file to test WAIC between models

In [ ]:
bn_area_age2 = CmdStanModel(stan_file=("neg binom area.stan"))
bn_edge_age2 = CmdStanModel(stan_file=("neg binom edge.stan"))

### C.2 Run each model

In [ ]:
nb_samples = bn_area_age2.sample(
    data=MinMax_obs_and_covs_dict,
    parallel_chains=4,
    iter_warmup = 3000,
    iter_sampling = 3000,
    show_console=False)

### C.3 Save inference data for reproducibility
Repeat for each bird, truncation distance, and area vs edge.

1. Check r-hat with model.summery()['R_hat']
2. Check convergence with arviz.plot_trace()

In [ ]:
# EDGE MODEL with DIST TO FOREST
nb_150_idata = az.from_cmdstanpy(
    posterior= nb_samples,
    observed_data={"M": MinMax_obs_and_covs_dict["M"]},
    constant_data={"age": MinMax_obs_and_covs_dict["age"],
                   "age2": MinMax_obs_and_covs_dict["age2"],
                   "edge" : MinMax_obs_and_covs_dict['edge'],
                   "patch" :MinMax_obs_and_covs_dict['patch'],
                   "dist_forest" : MinMax_obs_and_covs_dict['dist_forest'],
                   "tree_groups": MinMax_obs_and_covs_dict['tree_groups']},
    coords= {"beta_size_dim_0" : ["Pine", "Deciduous", "Mixed", "Spruce"],
            "beta_age_dim_0" : ["Pine", "Deciduous", "Mixed", "Spruce"]},
    log_likelihood="log_lik"
)
nb_150_idata.rename_vars({"beta_edge": "Patch edge", "beta_dist_forest": "Distance to forest", "beta_patch":"Patch", "beta_age_patch": "Patch:Age", "beta_age": "Age", "beta_age2": "Age${^2}$", "alpha": "Intercept"}, inplace=True)
nb_150_idata.to_netcdf(f'{bird}', f'{bird}_150_nb.nc')

# D. Plot posterior predictive checks

### D.1 Compare loo
To test whether applying truncation leads to better predictive data

In [ ]:
model_compare = az.compare(compare_dict={'unlimited': nb_unlimited_idata, 
                         '250m': nb_250_idata, 
                         '150m': nb_150_idata},
                            ic = 'loo')

### D.2 Compare WAIC
To choose the model

In [ ]:
comparison_waic = az.compare({'edge': model_with_edge,
            'area': model_with_area,
            'edge_area': model_with_edge_area,
            'dist_to_forest': model_with_forest_dist,
            'age2_area': model_with_area_age2,
            'age2_edge': model_with_edge_age2},
            ic = "waic")